# **2.5 Calculating Atomic Contributions for Molecules Based on a Graph Convolutional QSAR Model**

In [1]:
!pip install --pre deepchem

In [2]:
import deepchem
import warnings
warnings.filterwarnings('ignore')
deepchem.__version__

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.1.dev'

원자별 기여도 구하기.

전체 분자의 예측값 구함 -> 특정원자 하나 제거하고 예측값 구함.

이 차이 = 기여도

기여도로 얼만큼 중요한지 확인


In [5]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3457  100  3457    0     0  54724      0 --:--:-- --:--:-- --:--:-- 54873


In [6]:
import os
import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf

current_dir = os.path.dirname(os.path.realpath('__file__'))
dc.utils.download_url(
    'https://raw.githubusercontent.com/deepchem/deepchem/master/examples/tutorials/assets/atomic_contributions_tutorial_data/logBB.sdf',
    current_dir,
    'logBB.sdf'
)
DATASET_FILE =os.path.join(current_dir, 'logBB.sdf')
# Create RDKit mol objects, since we will need them later.
mols = [m for m in Chem.SDMolSupplier(DATASET_FILE) if m is not None ]
loader = dc.data.SDFLoader(tasks=["logBB_class"],
                           featurizer=dc.feat.ConvMolFeaturizer(),
                           sanitize=True)
dataset = loader.create_dataset(DATASET_FILE, shard_size=2000)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:02:19] DEPR

In [7]:
np.random.seed(2020)
tf.random.set_seed(2020)

In [ ]:
m = dc.models.GraphConvModel(1, mode="classification", batch_normalize=False, batch_size=100)
m.fit(dataset, nb_epoch=10)

In [10]:
# A fragment dataset
# Now let's prepare a dataset of fragments based on the training set. (Any other unseen data set of interest can also be used). These fragments will be used to evaluate the contributions of individual atoms.

# For each molecule we will generate a list of ConvMol objects. Specifying per_atom_fragmentation=True tells it to iterate over all heavy atoms and featurize a single-atom-depleted version of the molecule with each one removed.

loader = dc.data.SDFLoader(tasks=[],# dont need task (moreover, passing the task can lead to inconsitencies in data shapes)
                        featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=True),
                        sanitize=True)
frag_dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)
# /usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
#   return array(a, dtype, copy=False, order=order)
# /usr/local/lib/python3.7/dist-packages/deepchem/data/data_loader.py:885: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
#   return np.array(features), valid_inds
# The dataset still has the same number of samples as the original training set, but each sample is now represented as a list of ConvMol objects (one for each fragment) rather than a single ConvMol.

# IMPORTANT: The order of fragments depends on the input format. If SDF, the fragment order is the same as the atom order in corresponding mol blocks. If SMILES (i.e. csv with molecules represented as SMILES), then the order is given by RDKit CanonicalRankAtoms

print(frag_dataset.X.shape)
# (298,)
# We really want to treat each fragment as a separate sample. We can use a FlatteningTransformer to flatten the fragments lists.

tr = dc.trans.FlatteningTransformer(frag_dataset)
frag_dataset = tr.transform(frag_dataset)
print(frag_dataset.X.shape)
# (5111,)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPRECATION WARNING: please use GetValence(getExplicit=False)
[13:04:06] DEPR

In [11]:
print(frag_dataset.X.shape)

(298,)


In [12]:
tr = dc.trans.FlatteningTransformer(frag_dataset)
frag_dataset = tr.transform(frag_dataset)
print(frag_dataset.X.shape)

(5111,)


원자를 하나씩 빼가며 불완전한 분자 생

In [ ]:
# whole  molecules
pred = np.squeeze(m.predict(dataset))[:, 1] # probabilitiy of class 1
pred = pd.DataFrame(pred, index=dataset.ids, columns=["Molecule"])  # turn to dataframe for convinience

# fragments
pred_frags = np.squeeze(m.predict(frag_dataset))[:, 1]
pred_frags = pd.DataFrame(pred_frags, index=frag_dataset.ids, columns=["Fragment"])

In [ ]:
# merge 2 dataframes by molecule names
df = pd.merge(pred_frags, pred, right_index=True, left_index=True)
# find contribs
df['Contrib'] = df["Molecule"] - df["Fragment"]

In [ ]:
df

In [ ]:
def vis_contribs(mols, df, smi_or_sdf = "sdf"):
    # input format of file, which was used to create dataset determines the order of atoms,
    # so we take it into account for correct mapping!
    maps = []
    for mol  in mols:
        wt = {}
        if smi_or_sdf == "smi":
            for n,atom in enumerate(Chem.rdmolfiles.CanonicalRankAtoms(mol)):
                wt[atom] = df.loc[mol.GetProp("_Name"),"Contrib"][n]
        if smi_or_sdf == "sdf":
            for n,atom in enumerate(range(mol.GetNumHeavyAtoms())):
                wt[atom] = df.loc[Chem.MolToSmiles(mol),"Contrib"][n]
        maps.append(SimilarityMaps.GetSimilarityMapFromWeights(mol,wt))
    return maps

In [ ]:
np.random.seed(2000)
maps = vis_contribs(np.random.choice(np.array(mols),10), df)

위에서 말한 계산식으로 기여도 계산

SimilarityMaps를 이용해 분자 구조 위에 색으로 입

In [ ]:
current_dir = os.path.dirname(os.path.realpath('__file__'))
dc.utils.download_url(
    'https://raw.githubusercontent.com/deepchem/deepchem/master/examples/tutorials/assets/atomic_contributions_tutorial_data/Tetrahymena_pyriformis_Work_set_OCHEM.sdf',
    current_dir,
    'Tetrahymena_pyriformis_Work_set_OCHEM.sdf'
)
DATASET_FILE =os.path.join(current_dir, 'Tetrahymena_pyriformis_Work_set_OCHEM.sdf')

# create RDKit mol objects, we will need them later
mols = [m for m in Chem.SDMolSupplier(DATASET_FILE) if m is not None ]
loader = dc.data.SDFLoader(tasks=["IGC50"],
                           featurizer=dc.feat.ConvMolFeaturizer(), sanitize=True)
dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)

In [ ]:
np.random.seed(2020)
tf.random.set_seed(2020)
m = dc.models.GraphConvModel(1, mode="regression", batch_normalize=False)
m.fit(dataset, nb_epoch=40)

분류 vs 회귀

분류는 확률의 변화량  회기는 수치 그 자체의 변화

In [ ]:
current_dir = os.path.dirname(os.path.realpath('__file__'))
dc.utils.download_url(
    'https://raw.githubusercontent.com/deepchem/deepchem/master/examples/tutorials/assets/atomic_contributions_tutorial_data/Tetrahymena_pyriformis_Test_set_OCHEM.sdf',
    current_dir,
    'Tetrahymena_pyriformis_Test_set_OCHEM.sdf'
)




TEST_DATASET_FILE = os.path.join(current_dir, 'Tetrahymena_pyriformis_Test_set_OCHEM.sdf')
loader = dc.data.SDFLoader(tasks=["IGC50"], sanitize= True,
                           featurizer=dc.feat.ConvMolFeaturizer())
test_dataset = loader.create_dataset(TEST_DATASET_FILE, shard_size=2000)
pred = m.predict(test_dataset)
mse = metrics.mean_squared_error(y_true=test_dataset.y, y_pred=pred)
r2 = metrics.r2_score(y_true=test_dataset.y, y_pred=pred)
print(mse)
print(r2)

In [ ]:
loader = dc.data.SDFLoader(tasks=[], # dont need any task
                           sanitize=True,
                           featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=True))
frag_dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)
tr = dc.trans.FlatteningTransformer(frag_dataset) # flatten dataset and add ids to each fragment
frag_dataset = tr.transform(frag_dataset)

In [ ]:
# whole molecules
pred = m.predict(dataset)
pred = pd.DataFrame(pred, index=dataset.ids, columns=["Molecule"])  # turn to dataframe for convenience

# fragments
pred_frags = m.predict(frag_dataset)
pred_frags = pd.DataFrame(pred_frags, index=frag_dataset.ids, columns=["Fragment"])  # turn to dataframe for convenience

# merge 2 dataframes by molecule names
df = pd.merge(pred_frags, pred, right_index=True, left_index=True)
# find contribs
df['Contrib'] = df["Molecule"] - df["Fragment"]

In [ ]:
maps = vis_contribs([mol for mol in mols if float(mol.GetProp("IGC50"))>3 and float(mol.GetProp("IGC50"))<4][:10], df)

CSV/SMILES파일은 내부적으로 읽어들일 때 다시 정렬하는 경우가 많음

CanonicalRankAtoms 분자의 구조를 보고 표준적인 순서를 매겨줌

어떤 경로로 읽던 동일한 분자면 항상 똑같은 원자 번호를 부여하도록 강제

# 💡 2.5 Atomic Contributions 핵심 Insight

블랙박스 해소: 분자에서 원자를 하나씩 제거하며 예측값의 변화를 측정하는 '오클루전(Occlusion)' 기법을 통해, 모델이 판단 근거로 삼은 원자 단위의 기여도를 정량적으로 추출할 수 있음.

화학적 타당성 검증: 시각화된 Similarity Map을 통해 모델이 독성 원자(Toxicophores)나 투과 촉진 구조를 제대로 짚고 있는지 확인하고, 이를 바탕으로 실험의 시행착오를 줄이는 소재 설계 가이드라인을 확보함.

데이터 정합성의 디테일: 단순 예측을 넘어 원자 번호 하나까지 정확히 매칭하는 Canonical Ranking 등의 엔지니어링 처리가 뒷받침되어야 AI의 해석을 실제 합성 및 공정 현장에 신뢰감 있게 적용할 수 있음.